In [1]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import theano
import theano.tensor as tt
from theano.tensor import fft
import theano.sparse

import pymc3 as pm

# add the gridding path 
import sys
sys.path.append("/home/ian/Research/Disks/MillionPoints/million-points-of-light")

import gridding

# convert from arcseconds to radians
arcsec = np.pi / (180.0 * 3600) # [radians]  = 1/206265 radian/arcsec

In [2]:
def sky_plane(alpha, dec, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    '''
    alpha: ra (in radians)
    delta: dec (in radians)
    a : amplitude
    delta_alpha : offset (in radians)
    delta_dec : offset (in radians)
    sigma_alpha : width (in radians)
    sigma_dec : width (in radians)
    Omega : position angle of ascending node (in degrees east of north)
    '''

    return a * np.exp(-( (alpha - delta_alpha)**2/(2 * sigma_alpha**2) + \
                        (dec - delta_delta)**2/(2 * sigma_delta**2)))


def fourier_plane(u, v, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    '''
    Calculate the Fourier transform of the Gaussian. Assumes u, v in kλ.
    '''

    # convert back to lambda
    u = u * 1e3
    v = v * 1e3

    return 2 * np.pi * a * sigma_alpha * sigma_delta * np.exp(- 2 * np.pi**2 * \
                (sigma_alpha**2 * u**2 + sigma_delta**2 * v**2) - 2 * np.pi * 1.0j * \
                                                    (delta_alpha * u + delta_delta * v))


# the gradients
def dV_ddelta_alpha(u, v, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    
    
    return -2 * np.pi * 1j * u * fourier_plane(u*1e-3, v*1e-3, a, delta_alpha, delta_delta, sigma_alpha,
              sigma_delta, Omega)


def dV_ddelta_delta(u, v, a=1, delta_alpha=1.0*arcsec, delta_delta=1.0*arcsec, sigma_alpha=1.0*arcsec,
              sigma_delta=1.0*arcsec, Omega=0.0):
    
    
    return -2 * np.pi * 1j * v * fourier_plane(u*1e-3, v*1e-3, a, delta_alpha, delta_delta, sigma_alpha,
              sigma_delta, Omega)


In [3]:
def fftspace(width, N):
    '''Oftentimes it is necessary to get a symmetric coordinate array that spans ``N``
     elements from `-width` to `+width`, but makes sure that the middle point lands
     on ``0``. The indices go from ``0`` to ``N -1.``
     `linspace` returns  the end points inclusive, wheras we want to leave out the
     right endpoint, because we are sampling the function in a cyclic manner.'''

    assert N % 2 == 0, "N must be even."

    dx = width * 2.0 / N
    xx = np.empty(N, np.float)
    for i in range(N):
        xx[i] = -width + i * dx
    
    return xx

In [4]:
# Let's plot this up and see what it looks like 

N_alpha = 128
N_dec = 128
img_radius = 15.0 * arcsec


# full span of the image
ra = fftspace(img_radius, N_alpha) # [arcsec]
dec = fftspace(img_radius, N_dec) # [arcsec]

In [5]:
# calculate the maximum u and v points that our image grid can sample 
dRA = (2 * img_radius) / N_alpha # radians
max_baseline = 1 / (2 * dRA) * 1e-3 # kilolambda, nyquist rate
print(max_baseline) # kilolambda

440.03158666047227


In [6]:
# create some fake data

N_vis = 100 # number of data points 

# the fake baselines where the Visibility function is sampled 
u_data = np.random.normal(loc=0, scale=0.1 * max_baseline, size=N_vis)
v_data = np.random.normal(loc=0, scale=0.1 * max_baseline, size=N_vis)

data_points = np.array([u_data, v_data]).T

# create a dataset of a Gaussian
data_only = fourier_plane(u_data, v_data)

# add some noise
noise = 1e-12 * np.ones(N_vis) # Jy
noise_draw = np.random.normal(loc=0, scale=noise, size=(N_vis)) + \
    np.random.normal(loc=0, scale=noise, size=(N_vis)) * 1.0j 
data_values = data_only + noise_draw

In [7]:
# create fixed quantities that we can pre-calculate in numpy before stuffing into the Theano part 

# the image plane grid (fixed throughout the problem)
XX, YY = np.meshgrid(np.fft.fftshift(ra), np.fft.fftshift(dec))

# the image-plane taper (fixed throughout problem)
corrfun = gridding.corrfun_mat(np.fft.fftshift(ra), np.fft.fftshift(dec))

# the u and v coordinates of the RFFT output (also fixed throughout problem)
us = np.fft.rfftfreq(N_alpha, d=(2 * img_radius)/N_alpha) * 1e-3  # convert to [kλ]
vs = np.fft.fftfreq(N_dec, d=(2 * img_radius)/N_dec) * 1e-3  # convert to [kλ]

# calculate the C_real and C_imag matrices
# these are scipy csc sparse matrices that will be stuffed into Theano objects
C_real, C_imag = gridding.calc_matrices(data_points, us, vs)

# Sampling in PyMC3 

In [28]:
# NOTE that these must be `fftshifted` already.
# add an extra dimension for the later packing into the rfft
alpha = XX[np.newaxis,:]
dalpha = np.abs(alpha[0,0,1] - alpha[0,0,0])
delta = YY[np.newaxis,:]
ddelta = np.abs(delta[0,1,0] - delta[0,0,0])

real_data = np.real(data_values)
imag_data = np.imag(data_values)

with pm.Model() as model:
    
    # Define the PyMC3 model parameters, which are just for the image plane model
    a = pm.Uniform("a", lower=0.0, upper=10.0)
    delta_alpha = pm.Uniform("delta_alpha", lower=-1*arcsec, upper=2*arcsec)
    delta_delta = pm.Uniform("delta_delta", lower=-1*arcsec, upper=2*arcsec)
    sigma_alpha = pm.Uniform("sigma_alpha", lower=0.5*arcsec, upper=1.5*arcsec)
    sigma_delta = pm.Uniform("sigma_delta", lower=0.5*arcsec, upper=1.5*arcsec)
    
    # Calculate the sky-plane model
    I = a * tt.exp(-(alpha - delta_alpha)**2/(2 * sigma_alpha**2) - (delta - delta_delta)**2/(2 * sigma_delta**2))
    # since the input coordinates were already shifted, then this is too
    # I shape should be (1, N_dec, N_alpha)

    # taper the image with the gridding correction function
    # this should broadcast OK, since the trailing image dimensions match
    I_tapered = I * corrfun

    # output from the RFFT is (1, N_delta, N_alpha//2 + 1, 2)
    rfft = dalpha * ddelta * fft.rfft(I_tapered, norm=None)  

    # flatten the RFFT output appropriately for the interpolation, taking the real and imag parts separately
    vis_real = rfft[0, :, :, 0].flatten() # real values 
    vis_imag = rfft[0, :, :, 1].flatten() # imaginary values

    # interpolate the RFFT to the baselines by a sparse matrix multiply
    interp_real = theano.sparse.dot(C_real, vis_real)
    interp_imag = theano.sparse.dot(C_imag, vis_imag)
    
    real_print = tt.printing.Print('interp_real')(interp_real)
    imag_print = tt.printing.Print('interp_imag')(interp_imag)
    
    # condition on the real and imaginary observations
    pm.Normal("obs_real", mu=interp_real, sd=noise, observed=real_data)
    pm.Normal("obs_imag", mu=interp_imag, sd=noise, observed=imag_data)

interp_real __str__ = [ 2.43649153e-11  2.91783491e-10  4.78838315e-10  5.97334182e-11
  1.04422700e-10  3.00205770e-10  8.39898986e-11  6.13498780e-10
  1.60080632e-10  5.30238815e-10  1.08821477e-10  1.09915574e-11
  2.43894596e-11  2.15971485e-10  1.40151900e-10  4.52815406e-10
  4.32711906e-13  4.60500344e-10  9.83354453e-12  5.98490733e-10
  2.65596472e-10  2.42823653e-10  2.05343116e-10  5.52404386e-11
  4.64253083e-10  3.35037160e-10  5.89143964e-11 -3.71065172e-13
  2.49523692e-10  2.51554188e-10  3.88052556e-11 -3.08989133e-13
  5.16683139e-10  2.08765871e-10  5.54311501e-10 -4.53432841e-13
 -9.93541790e-13  6.24293174e-12  2.61039411e-10  6.54590209e-10
  5.84436699e-12  7.34610572e-10  2.63060425e-10  4.68467386e-10
  2.33425881e-10  1.81211173e-10  1.17197276e-10  4.54781092e-10
 -2.75927279e-12  5.37373419e-14 -2.71850561e-14  2.77128187e-10
  7.06382364e-10  1.13526466e-10  1.11386630e-10  4.97543827e-10
  1.56218597e-10  3.34528319e-10 -3.03749664e-12  5.86349195e-10
  2

In [29]:
model.basic_RVs

[a_interval__,
 delta_alpha_interval__,
 delta_delta_interval__,
 sigma_alpha_interval__,
 sigma_delta_interval__,
 obs_real,
 obs_imag]

In [30]:
model.unobserved_RVs

[a_interval__,
 delta_alpha_interval__,
 delta_delta_interval__,
 sigma_alpha_interval__,
 sigma_delta_interval__,
 a,
 delta_alpha,
 delta_delta,
 sigma_alpha,
 sigma_delta]

In [31]:
model.free_RVs

[a_interval__,
 delta_alpha_interval__,
 delta_delta_interval__,
 sigma_alpha_interval__,
 sigma_delta_interval__]

In [36]:
with model:
    s = pm.Metropolis(vars=model.free_RVs)
    trace = pm.sample(100, tune=100, step=s, cores=1)

Only 100 samples in chain.
Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [sigma_delta]
>Metropolis: [sigma_alpha]
>Metropolis: [delta_delta]
>Metropolis: [delta_alpha]
>Metropolis: [a]
100%|██████████| 200/200 [00:01<00:00, 102.44it/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some parameters.


In [37]:
pm.summary(trace)

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
a,1.211181,9.401905e-02,9.380198e-03,1.105795,1.338733,1.214449,2.806077e+00
delta_alpha,0.000005,2.778387e-08,2.749910e-09,0.000005,0.000005,5.535205,1.024958e+00
delta_delta,0.000005,1.304752e-08,1.304752e-09,0.000005,0.000005,1.005025,1.370851e+13
sigma_alpha,0.000005,1.512093e-07,1.502771e-08,0.000004,0.000005,1.659639,1.853140e+00
sigma_delta,0.000004,2.747023e-07,2.746380e-08,0.000004,0.000004,1.107158,3.893290e+00


In [32]:
with model:
    trace = pm.sample(draws=1000, tune=1000, chains=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


ValueError: shapes (100,) and (8320,) not aligned: 100 (dim 0) != 8320 (dim 0)